In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configurar seed para reproducibilidad
np.random.seed(42)

In [2]:
# Cargar el dataset de vinos tintos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine_data = pd.read_csv(url, sep=';')

print("Dataset cargado exitosamente!")
print(f"Dimensiones del dataset: {wine_data.shape}")
print(f"\nColumnas del dataset:")
print(wine_data.columns.tolist())

Dataset cargado exitosamente!
Dimensiones del dataset: (1599, 12)

Columnas del dataset:
['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


In [3]:
# Mostrar las primeras filas del dataset
wine_data.head()

# TODO: Muestra la información general del dataset (tipos de datos, valores no nulos)
print("Información general del dataset:")
wine_data.info()
print("-" * 50)
# TODO: Calcula y muestra las estadísticas descriptivas del dataset
print("Estadísticas descriptivas del dataset:")
print(wine_data.describe().T)
print("-" * 50)
# TODO: Verifica si hay valores nulos en el dataset
print("Valores nulos por columna:")
print(wine_data.isnull().sum())
print("-" * 50)

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
--------------------------------------------------
Estadísticas descriptivas del dataset:
   

In [ ]:
# Analizar la distribución de la calidad del vino
plt.figure(figsize=(10, 6))
wine_data['quality'].value_counts().sort_index().plot(kind='bar', color='steelblue', edgecolor='black')
plt.xlabel('Calidad del Vino', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Distribución de la Calidad del Vino', fontsize=14)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

# Agregar estadísticas
mean_quality = wine_data['quality'].mean()
median_quality = wine_data['quality'].median()
plt.axhline(y=wine_data['quality'].value_counts().mean(), color='red', 
            linestyle='--', label=f'Media de frecuencia')

plt.legend()
plt.tight_layout()
plt.show()

print(f"Estadísticas de la calidad del vino:")
print(f"Media: {mean_quality:.2f}")
print(f"Mediana: {median_quality:.2f}")
print(f"Desviación estándar: {wine_data['quality'].std():.2f}")

In [ ]:
# TODO: Calcula la matriz de correlación y visualízala con un heatmap
# Pista: Usa sns.heatmap() con annot=True para mostrar los valores
matriz_corr = wine_data.corr()

plt.figure(figsize=(14, 10))
# Completa el código para crear el heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", 
            linewidths=.5, cbar_kws={'label': 'Coeficiente de Correlación'})
plt.title('Matriz de Correlación del Dataset de Vinos', fontsize=16)
plt.show()

# TODO: Identifica y muestra las 5 variables más correlacionadas con 'quality'
print("-" * 50)
print("Variables más correlacionadas con la calidad (quality):")
print(correlation_matrix['quality'].sort_values(ascending=False).head(6)[1:])

In [ ]:
# Visualizar las 4 variables más correlacionadas con quality
top_features = ['alcohol', 'volatile acidity', 'citric acid', 'sulphates']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Relación entre Variables Principales y Calidad del Vino', fontsize=16)

for idx, (ax, feature) in enumerate(zip(axes.flat, top_features)):
    # TODO: Crea un scatter plot para cada variable vs quality
    sns.scatterplot(x=wine_data[feature], y=wine_data['quality'], ax=ax, alpha=0.6, color='b')
    # Agrega una línea de tendencia
    sns.regplot(x=wine_data[feature], y=wine_data['quality'], ax=ax, scatter=False, color='red', line_kws={'linestyle': '--'})
    ax.set_title(f'Calidad vs. {feature.replace("_", " ").capitalize()}', fontsize=12)
    ax.set_xlabel(feature.replace("_", " ").capitalize(), fontsize=10)
    ax.set_ylabel('Calidad del Vino', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.5)
    pass

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Separar características (X) y variable objetivo (y)
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"\nCaracterísticas: {X.columns.tolist()}")

In [ ]:
# TODO: Divide los datos en conjuntos de entrenamiento y prueba
# Usa test_size=0.2 y random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nDivisión de los datos:")
print(f"Tamaño del conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Tamaño del conjunto de prueba (X_test): {X_test.shape}")
print(f"Tamaño de las etiquetas de entrenamiento (y_train): {y_train.shape}")
print(f"Tamaño de las etiquetas de prueba (y_test): {y_test.shape}")

In [ ]:
# TODO: Estandariza las características
# Recuerda: ajusta el scaler solo con los datos de entrenamiento
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

print("Estandarización de características completada.")
print(f"Forma del conjunto de entrenamiento escalado: {X_train_scaled.shape}")
print(f"Forma del conjunto de prueba escalado: {X_test_scaled.shape}")

In [ ]:
# Ejemplo: Validación cruzada manual para Ridge
def manual_cross_validation(X, y, alpha, n_folds=5):
    """
    Implementa validación cruzada manualmente para Ridge regression
    """
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in kf.split(X):
        # Dividir datos
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
        
        # Estandarizar
        scaler_cv = StandardScaler()
        X_train_cv_scaled = scaler_cv.fit_transform(X_train_cv)
        X_val_cv_scaled = scaler_cv.transform(X_val_cv)
        
        # Entrenar modelo
        model = Ridge(alpha=alpha)
        model.fit(X_train_cv_scaled, y_train_cv)
        
        # Evaluar
        y_pred = model.predict(X_val_cv_scaled)
        rmse = np.sqrt(mean_squared_error(y_val_cv, y_pred))
        scores.append(rmse)
    
    return np.mean(scores), np.std(scores)

# Probar diferentes valores de alpha
alphas_to_test = [0.001, 0.01, 0.1, 1, 10, 100]
cv_results_manual = []

print("Validación Cruzada Manual para Ridge Regression:")
print("-" * 50)

for alpha in alphas_to_test:
    mean_rmse, std_rmse = manual_cross_validation(X_train.values, y_train, alpha)
    cv_results_manual.append({'alpha': alpha, 'mean_rmse': mean_rmse, 'std_rmse': std_rmse})
    print(f"Alpha: {alpha:7.3f} | RMSE: {mean_rmse:.4f} (+/- {std_rmse:.4f})")

# TODO: Identifica el mejor alpha basado en el RMSE medio más bajo
if cv_results_manual:
    best_alpha_manual = min(cv_results_manual, key=lambda x: x['mean_rmse'])
    print("-" * 50)
    print("Resultados de la validación cruzada manual:")
    print(pd.DataFrame(cv_results_manual).sort_values(by='mean_rmse'))
    print(f"\nEl mejor valor de alpha encontrado manualmente es: {best_alpha_manual['alpha']:.3f} (RMSE: {best_alpha_manual['mean_rmse']:.4f})")

In [ ]:
# TODO: Usa RidgeCV para encontrar automáticamente el mejor alpha
# Pista: RidgeCV tiene un parámetro 'alphas' y 'cv'

alphas = np.logspace(-3, 3, 100)  # 100 valores entre 0.001 y 1000
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_root_mean_squared_error')
ridge_cv.fit(X_train_scaled, y_train)

print(f"Mejor alpha encontrado por RidgeCV: {ridge_cv.alpha_:.4f}")
print(f"RMSE (Validación Cruzada) para el mejor alpha: {np.sqrt(-ridge_cv.best_score_):.4f}")

In [ ]:
# Ejemplo completo con GridSearchCV para Ridge
from sklearn.model_selection import GridSearchCV

# Definir parámetros a buscar
param_grid_ridge = {
    'alpha': np.logspace(-3, 3, 20)  # 20 valores entre 0.001 y 1000
}

# Crear modelo base
ridge_base = Ridge(random_state=42)

grid_search_ridge = GridSearchCV(
    estimator=ridge_base,
    param_grid=param_grid_ridge,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search_ridge.fit(X_train_scaled, y_train)
best_rmse = np.sqrt(-grid_search_ridge.best_score_)

print("\nBúsqueda con GridSearchCV completada.")
print("-" * 50)
print(f"Mejor alpha: {grid_search_ridge.best_params_['alpha']:.4f}")
print(f"Mejor score (RMSE): {best_rmse:.4f}")
print("-" * 50)
print("Resultados completos de la validación cruzada:")

results_df = pd.DataFrame(grid_search_ridge.cv_results_)[['param_alpha', 'mean_test_score', 'std_test_score']]
results_df['mean_rmse'] = np.sqrt(-results_df['mean_test_score'])
print(results_df.sort_values(by='mean_rmse').head())

In [ ]:
# TODO: Visualiza los resultados de la validación cruzada
# Crea un gráfico que muestre cómo cambia el RMSE con diferentes valores de alpha
results_df = pd.DataFrame(grid_search_ridge.cv_results_)
results_df['mean_rmse'] = np.sqrt(-results_df['mean_test_score'])
results_df['std_rmse'] = results_df['std_test_score'] / (2 * np.sqrt(len(y_train))) # Cálculo del error estándar del RMSE

plt.figure(figsize=(10, 6))
plt.errorbar(
    results_df['param_alpha'],
    results_df['mean_rmse'],
    yerr=results_df['std_rmse'],
    fmt='-o',
    capsize=3,
    elinewidth=1,
    label='RMSE Promedio (+/- error estándar)',
    color='steelblue'
)

plt.xscale('log')
plt.xlabel('Alpha (parámetro de regularización)', fontsize=12)
plt.ylabel('RMSE de Validación Cruzada', fontsize=12)
plt.title('RMSE de Validación Cruzada vs. Alpha para Ridge Regression', fontsize=14)
plt.axvline(x=grid_search_ridge.best_params_['alpha'], color='red', linestyle='--', label=f"Mejor Alpha: {grid_search_ridge.best_params_['alpha']:.3f}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Implementa y entrena un modelo de regresión lineal
# Tu código aquí

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred_lr_train = lr_model.predict(X_train_scaled)
y_pred_lr_test = lr_model.predict(X_test_scaled)

# Calcular métricas
rmse_lr_train = np.sqrt(mean_squared_error(y_train, y_pred_lr_train))
rmse_lr_test = np.sqrt(mean_squared_error(y_test, y_pred_lr_test))
r2_lr_train = r2_score(y_train, y_pred_lr_train)
r2_lr_test = r2_score(y_test, y_pred_lr_test)

print("Regresión Lineal Normal:")
print("-" * 50)
print(f"RMSE de Entrenamiento: {rmse_lr_train:.4f}")
print(f"RMSE de Prueba:        {rmse_lr_test:.4f}")
print(f"R² de Entrenamiento:   {r2_lr_train:.4f}")
print(f"R² de Prueba:          {r2_lr_test:.4f}")

In [ ]:
# TODO: Entrena Ridge con el mejor alpha encontrado por validación cruzada

best_alpha_ridge = 9.486832980892942
ridge_model = Ridge(alpha=best_alpha_ridge, random_state=42)
ridge_model.fit(X_train_scaled, y_train)

# Predicciones y métricas
y_pred_ridge_train = ridge_model.predict(X_train_scaled)
y_pred_ridge_test = ridge_model.predict(X_test_scaled)

rmse_ridge_train = np.sqrt(mean_squared_error(y_train, y_pred_ridge_train))
rmse_ridge_test = np.sqrt(mean_squared_error(y_test, y_pred_ridge_test))
r2_ridge_train = r2_score(y_train, y_pred_ridge_train)
r2_ridge_test = r2_score(y_test, y_pred_ridge_test)

print("Ridge Regression con mejor alpha de CV:")
print("-" * 50)
print(f"Mejor Alpha utilizado: {best_alpha_ridge:.4f}")
print(f"RMSE de Entrenamiento: {rmse_ridge_train:.4f}")
print(f"RMSE de Prueba:        {rmse_ridge_test:.4f}")
print(f"R² de Entrenamiento:   {r2_ridge_train:.4f}")
print(f"R² de Prueba:          {r2_ridge_test:.4f}")

In [ ]:
# TODO: Implementa LassoCV para encontrar el mejor alpha automáticamente
alphas_lasso = np.logspace(-3, 1, 100)
lasso_cv = LassoCV(alphas=alphas_lasso, cv=5, random_state=42, n_jobs=-1)
lasso_cv.fit(X_train_scaled, y_train)

# Predicciones y métricas
y_pred_lasso_train = lasso_cv.predict(X_train_scaled)
y_pred_lasso_test = lasso_cv.predict(X_test_scaled)

rmse_lasso_train = np.sqrt(mean_squared_error(y_train, y_pred_lasso_train))
rmse_lasso_test = np.sqrt(mean_squared_error(y_test, y_pred_lasso_test))
r2_lasso_train = r2_score(y_train, y_pred_lasso_train)
r2_lasso_test = r2_score(y_test, y_pred_lasso_test)

num_selected_features = np.sum(lasso_cv.coef_ != 0)

print("Lasso Regression con Validación Cruzada:")
print("-" * 50)
print(f"Mejor alpha para Lasso: {lasso_cv.alpha_:.4f}")
print(f"Número de características seleccionadas: {num_selected_features}")
print(f"RMSE de Entrenamiento: {rmse_lasso_train:.4f}")
print(f"RMSE de Prueba:        {rmse_lasso_test:.4f}")
print(f"R² de Entrenamiento:   {r2_lasso_train:.4f}")
print(f"R² de Prueba:          {r2_lasso_test:.4f}")

# TODO: Identifica qué características fueron eliminadas por Lasso
eliminated_features = feature_names[lasso_cv.coef_ == 0]
selected_features = feature_names[lasso_cv.coef_ != 0]

print("Análisis de Coeficientes de Lasso:")
print("-" * 50)
print(f"Características eliminadas (coeficiente = 0):")
if len(eliminated_features) > 0:
    for feature in eliminated_features:
        print(f"- {feature}")
else:
    print("Ninguna característica fue eliminada por Lasso en este rango de alphas.")
print("-" * 50)
print(f"Características seleccionadas (coeficiente ≠ 0):")
print(f"{selected_features.tolist()}")
print("-" * 50)
print("Coeficientes de las características:")
lasso_coefficients = pd.DataFrame({'feature': feature_names, 'coefficient': lasso_cv.coef_})
print(lasso_coefficients.sort_values(by='coefficient', ascending=False))

In [ ]:
# TODO: Crea una tabla comparativa con todos los modelos
# Incluye: RMSE Train, RMSE Test, R² Train, R² Test, MAE Test

comparison_data = {
    'Modelo': ['Regresión Lineal', 'Ridge', 'Lasso'],
    'RMSE Train': [
        np.sqrt(mean_squared_error(y_train, y_pred_lr_train)),
        np.sqrt(mean_squared_error(y_train, y_pred_ridge_train)),
        np.sqrt(mean_squared_error(y_train, y_pred_lasso_train))
    ],
    'RMSE Test': [
        np.sqrt(mean_squared_error(y_test, y_pred_lr_test)),
        np.sqrt(mean_squared_error(y_test, y_pred_ridge_test)),
        np.sqrt(mean_squared_error(y_test, y_pred_lasso_test))
    ],
    'R² Train': [
        r2_score(y_train, y_pred_lr_train),
        r2_score(y_train, y_pred_ridge_train),
        r2_score(y_train, y_pred_lasso_train)
    ],
    'R² Test': [
        r2_score(y_test, y_pred_lr_test),
        r2_score(y_test, y_pred_ridge_test),
        r2_score(y_test, y_pred_lasso_test)
    ],
    'MAE Test': [
        mean_absolute_error(y_test, y_pred_lr_test),
        mean_absolute_error(y_test, y_pred_ridge_test),
        mean_absolute_error(y_test, y_pred_lasso_test)
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(4))

In [ ]:
# TODO: Crea visualizaciones para comparar los modelos
# 1. Gráfico de barras comparando RMSE
plt.figure(figsize=(10, 6))
rmse_plot = sns.barplot(x='Modelo', y='RMSE', data=performance_df, palette='viridis')
plt.title('Comparación de RMSE de los Modelos en el Conjunto de Prueba', fontsize=14)
plt.xlabel('Modelo', fontsize=12)
plt.ylabel('RMSE', fontsize=12)
plt.ylim(0.64, 0.66)

for index, row in performance_df.iterrows():
    rmse_plot.text(row.name, row.RMSE + 0.0005, f'{row.RMSE:.4f}', color='black', ha="center")

plt.tight_layout()
plt.show()

# 2. Gráfico de barras comparando R²
plt.figure(figsize=(10, 6))
r2_plot = sns.barplot(x='Modelo', y='R²', data=performance_df, palette='viridis')
plt.title('Comparación de R² de los Modelos en el Conjunto de Prueba', fontsize=14)
plt.xlabel('Modelo', fontsize=12)
plt.ylabel('R²', fontsize=12)
plt.ylim(0.35, 0.38) 

for index, row in performance_df.iterrows():
    r2_plot.text(row.name, row.R2 + 0.0005, f'{row.R2:.4f}', color='black', ha="center")

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Para el mejor modelo, crea:
# 1. Gráfico de residuos vs predicciones
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred_lasso_test, y=residuals, alpha=0.6, color='steelblue')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicciones', fontsize=12)
plt.ylabel('Residuos', fontsize=12)
plt.title('Gráfico de Residuos vs. Predicciones', fontsize=14)
plt.grid(True)
plt.show()

# 2. Histograma de residuos
plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=30, kde=True, color='steelblue', edgecolor='black')
plt.xlabel('Residuos', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Histograma de Residuos', fontsize=14)
plt.grid(True)
plt.show()

# 3. Q-Q plot de residuos
fig = sm.qqplot(residuals, line='45', fit=True)
fig.set_size_inches(10, 6)
plt.title('Q-Q Plot de Residuos', fontsize=14)
plt.show()

In [ ]:
# TODO: Visualiza los coeficientes de los tres modelos en un mismo gráfico
# Esto te ayudará a entender qué características son más importantes
coef_df_melted = coef_df.melt('Features', var_name='Modelo', value_name='Coeficiente')
plt.figure(figsize=(15, 8))
sns.barplot(x='Features', y='Coeficiente', hue='Modelo', data=coef_df_melted, palette='tab10')
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.axhline(y=0, color='black', linestyle='-', linewidth=1.5)
plt.title('Comparación de Coeficientes de los Modelos de Regresión', fontsize=16)
plt.xlabel('Características', fontsize=12)
plt.ylabel('Valor del Coeficiente', fontsize=12)
plt.legend(title='Modelo')
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Realiza validación cruzada con 10 folds del mejor modelo
# Reporta la media y desviación estándar del RMSE
final_lasso_model = Lasso(alpha=best_alpha_lasso, random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

cv_scores = cross_val_score(
    final_lasso_model,
    X_scaled,
    y,
    cv=kf,
    scoring=rmse_scorer,
    n_jobs=-1
)

# Convertir los scores negativos a RMSE
rmse_scores = np.sqrt(-cv_scores)

# Reportar los resultados
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print("Validación Cruzada Final del Modelo Lasso:")
print("-" * 50)
print(f"Alpha utilizado: {best_alpha_lasso:.4f}")
print(f"Media del RMSE (10 pliegues):    {mean_rmse:.4f}")
print(f"Desviación Estándar del RMSE: {std_rmse:.4f}")